Download the Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!kaggle datasets download -d bittlingmayer/amazonreviews


Dataset URL: https://www.kaggle.com/datasets/bittlingmayer/amazonreviews
License(s): unknown
 98% 482M/493M [00:04<00:00, 105MB/s]
100% 493M/493M [00:04<00:00, 118MB/s]


In [4]:
# Unzip the dataset
!unzip /content/drive/MyDrive/amazonreviews-dataset/amazonreviews.zip -d /content/drive/MyDrive/amazonreviews-dataset

Archive:  /content/drive/MyDrive/amazonreviews-dataset/amazonreviews.zip
  inflating: /content/drive/MyDrive/amazonreviews-dataset/test.ft.txt.bz2  
  inflating: /content/drive/MyDrive/amazonreviews-dataset/train.ft.txt.bz2  


Set Up Your Environment

In [2]:
!pip install transformers pandas


Load and Decompress the Dataset

In [ ]:
import pandas as pd
import bz2
from transformers import pipeline
from sklearn.metrics import classification_report

gdrive_dir = '/content/drive/MyDrive/amazonreviews-dataset/'
train_dir = gdrive_dir + 'train.ft.txt.bz2'
test_dir = gdrive_dir + 'test.ft.txt.bz2'

with bz2.open(train_dir, 'rt') as f:
    data = f.readlines()

# with bz2.BZ2File(train_dir, 'rb') as f:
#     data = f.read().decode('utf-8').splitlines()

# Check the total lines and the first few entries
print("Total lines in dataset:", len(data))
for line in data[:5]:
    print(line.strip())

# Prepare valid reviews
reviews = []
for line in data:
    # Split the line on the first space to separate the label from the review
    parts = line.strip().split(' ', 1)  # Split only on the first space
    if len(parts) == 2:  # Check if the line has the expected number of columns
        reviews.append(parts)

# Create DataFrame from valid reviews
df = pd.DataFrame(reviews, columns=['sentiment', 'reviewText'])

# Check the resulting DataFrame
print("DataFrame shape:", df.shape)
print("DataFrame head:")
print(df.head())

In [22]:
print(df.head())

    sentiment                                         reviewText
0  __label__2  Stuning even for the non-gamer: This sound tra...
1  __label__2  The best soundtrack ever to anything.: I'm rea...
2  __label__2  Amazing!: This soundtrack is my favorite music...
3  __label__2  Excellent Soundtrack: I truly like this soundt...
4  __label__2  Remember, Pull Your Jaw Off The Floor After He...


Preprocess the Data Clean and label the sentiment based on the dataset. The first column usually indicates the sentiment label.

In [23]:
# Check the unique sentiment labels
#print(df['sentiment'].unique())

# Map sentiments to a simpler format
sentiment_mapping = {
    '__label__1': 'negative',
    '__label__2': 'positive'
}

df['mapped_sentiment'] = df['sentiment'].map(sentiment_mapping)
print()


Load the Pre-trained Sentiment Analysis Model Use the nlptown/bert-base-multilingual-uncased-sentiment model for sentiment analysis.

In [2]:
from transformers import pipeline

# Load the sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Analyze Sentiment Use the model to classify the sentiment of the reviews.

In [ ]:

# Prepare the reviews for sentiment analysis
review_texts = df['reviewText'].tolist()
print(review_texts)

# Analyze sentiment
predictions = sentiment_pipeline(review_texts)

# Extract sentiment labels and scores
df['predicted_sentiment'] = [pred['label'] for pred in predictions]
df['predicted_score'] = [pred['score'] for pred in predictions]

# Define your labels explicitly if necessary
target_names = ['negative', 'positive']  # Modify based on your classes

# Evaluate the model
print("Mapped Sentiment:", df['mapped_sentiment'].unique())
print("Predicted Sentiment:", df['predicted_sentiment'].unique())
print(classification_report(df['mapped_sentiment'], df['predicted_sentiment'], target_names=target_names))

# Display the first few results
print(df[['reviewText', 'mapped_sentiment', 'predicted_sentiment', 'predicted_score']].head())


In [ ]:
# Save the results to a CSV file
reviews.to_csv('amazon_reviews_with_sentiment.csv', index=False)
